## Using LightGBM regressor 

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager

In [ ]:
N_SPLITS = 20
SEED = 1234

Get file path from kaggle 

In [ ]:
LOG_PATH = Path("./log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)

Some utility code

1. A timer
2. A seed setter
3. A intermediate metric finder
4. A score logger

In [ ]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df


Get all the data in our nb env

In [ ]:
set_seed(SEED)
feature_dir = "../input/indoor-navigation-and-location-wifi-features"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)

Create some tuned paratmeters for the LGB ( These may be tweaked to get the best performance)

In [ ]:
lgb_params_1 = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.01,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }


lgb_params_2 = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.01,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.02,
                'num_leaves': 90,
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }

Splitting the data into train and validation and passed them through LGBm and then ssaving the predicitions into pred

In [ ]:
import joblib
import pickle

In [ ]:
score_df = pd.DataFrame()

oof = list()

predictions = list()

for n_files, file in enumerate(train_files):
    
    if n_files != 19:
        
        continue
    
    data = pd.read_csv(file, index_col=0)
    
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]
        y_trainf = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        y_validf = data.iloc[val_idx, -2]

        modelx = lgb.LGBMRegressor(**lgb_params_1)
        
        #modelx.save_model('lgb_x'+f'_{fold}', num_iteration=modelx.best_iteration) 
        #modely.save_model('lgb_y'+f'_{fold}', num_iteration=modely.best_iteration) 
        #modelf.save_model('lgb_f'+f'_{fold}', num_iteration=modef.best_iteration) 
        
        with timer("fit X"):
            
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=250,
                       early_stopping_rounds=50
                       )

        modely = lgb.LGBMRegressor(**lgb_params_2)
        
        with timer("fit Y"):
            
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=250,
                       early_stopping_rounds=50
                       )
            
        modelf = lgb.LGBMClassifier(**lgb_f_params)
        
        with timer("fit F"):
            
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=250,
                       early_stopping_rounds=50
                       )
            
        plh_x = 'lgbx'+ '_' + str(fold) + '_' +'.pkl'
        plh_y = 'lgby'+ '_' + str(fold) + '_' +'.pkl'
        plh_f = 'lgbf'+ '_' + str(fold) + '_' +'.pkl'
            
        joblib.dump(modelx, plh_x)
        joblib.dump(modely, plh_y)
        joblib.dump(modelf, plh_f)

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-1]).astype(int)

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

Training until validation scores don't improve for 50 rounds
[250]	valid_0's rmse: 6.09353
[500]	valid_0's rmse: 3.35968
[750]	valid_0's rmse: 3.04734
[1000]	valid_0's rmse: 2.89739
[1250]	valid_0's rmse: 2.79644
[1500]	valid_0's rmse: 2.73825
[1750]	valid_0's rmse: 2.69349
[2000]	valid_0's rmse: 2.65794
[2250]	valid_0's rmse: 2.63244
[2500]	valid_0's rmse: 2.60824
[2750]	valid_0's rmse: 2.58642
Early stopping, best iteration is:
[2846]	valid_0's rmse: 2.5825


[0.5GB(+0.1GB): 35.854sec] fit X


Training until validation scores don't improve for 50 rounds
[250]	valid_0's rmse: 5.50205
[500]	valid_0's rmse: 3.43106
[750]	valid_0's rmse: 3.02702
[1000]	valid_0's rmse: 2.8613
[1250]	valid_0's rmse: 2.77004
[1500]	valid_0's rmse: 2.70445
[1750]	valid_0's rmse: 2.65588
[2000]	valid_0's rmse: 2.62825
[2250]	valid_0's rmse: 2.60135
[2500]	valid_0's rmse: 2.57954
[2750]	valid_0's rmse: 2.56159
Early stopping, best iteration is:
[2792]	valid_0's rmse: 2.55734


[0.5GB(+0.0GB): 34.362sec] fit Y


Training until validation scores don't improve for 50 rounds
[250]	valid_0's multi_logloss: 0.0583965
[500]	valid_0's multi_logloss: 0.0502752
[750]	valid_0's multi_logloss: 0.0488314
[1000]	valid_0's multi_logloss: 0.0478793
[1250]	valid_0's multi_logloss: 0.0470219
[1500]	valid_0's multi_logloss: 0.0466792
[1750]	valid_0's multi_logloss: 0.0462707
[2000]	valid_0's multi_logloss: 0.0459415
[2250]	valid_0's multi_logloss: 0.0456856
[2500]	valid_0's multi_logloss: 0.045264
[2750]	valid_0's multi_logloss: 0.0449907
[3000]	valid_0's multi_logloss: 0.0446813
[3250]	valid_0's multi_logloss: 0.0444745
[3500]	valid_0's multi_logloss: 0.0442799
[3750]	valid_0's multi_logloss: 0.0440772
[4000]	valid_0's multi_logloss: 0.0438043
Early stopping, best iteration is:
[3972]	valid_0's multi_logloss: 0.0438028


...

Training until validation scores don't improve for 50 rounds
[250]	valid_0's rmse: 5.37214
[500]	valid_0's rmse: 3.41029
[750]	valid_0's rmse: 3.02754
[1000]	valid_0's rmse: 2.87018
[1250]	valid_0's rmse: 2.78775
[1500]	valid_0's rmse: 2.73789
[1750]	valid_0's rmse: 2.70324
[2000]	valid_0's rmse: 2.67328
[2250]	valid_0's rmse: 2.65517
[2500]	valid_0's rmse: 2.64325
[2750]	valid_0's rmse: 2.63168
[3000]	valid_0's rmse: 2.61994
[3250]	valid_0's rmse: 2.61365
[3500]	valid_0's rmse: 2.60706
[3750]	valid_0's rmse: 2.60298
Early stopping, best iteration is:
[3716]	valid_0's rmse: 2.60235


[0.6GB(-0.0GB): 48.804sec] fit Y


Training until validation scores don't improve for 50 rounds
[250]	valid_0's multi_logloss: 0.0622501
[500]	valid_0's multi_logloss: 0.0539045
[750]	valid_0's multi_logloss: 0.0522154
[1000]	valid_0's multi_logloss: 0.0513948
[1250]	valid_0's multi_logloss: 0.0506624
[1500]	valid_0's multi_logloss: 0.049848
[1750]	valid_0's multi_logloss: 0.0495122
[2000]	valid_0's multi_logloss: 0.0492027
[2250]	valid_0's multi_logloss: 0.0489025
[2500]	valid_0's multi_logloss: 0.0485206
[2750]	valid_0's multi_logloss: 0.0481459
[3000]	valid_0's multi_logloss: 0.0478174
[3250]	valid_0's multi_logloss: 0.0476446
[3500]	valid_0's multi_logloss: 0.0474785
[3750]	valid_0's multi_logloss: 0.0473658
[4000]	valid_0's multi_logloss: 0.0471584
Early stopping, best iteration is:
[4008]	valid_0's multi_logloss: 0.0471509


[0.6GB(+0.0GB): 62.486sec] fit F


fold 19: mean position error 3.2412703692434484
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #19, shape=(6312, 1025), name=5da1389e4db8ce0c98bd0547_train.csv
mean position error 3.0676078798634983
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
